# Writing files with `splendaq`
----
In this notebook, we show how to write `splendaq` HDF5 files from generated data, useful for creating simulated data and analyzing it as if it were data from a DAQ system (in our case, a Moku).

In [ ]:
import numpy as np
from splendaq.io import Writer
import matplotlib.pyplot as plt
from datetime import datetime

Let's define the timescale that we will use for our simulated data. We'll use a digitization rate of 1.25e6 Hz, create a simulated trace of length 30 seconds, and add square pulses of with width of 0.8 ms.

In [ ]:
fs = 1.25e6 # [Hz]
tracelength = int(30 * fs) # 30 seconds [bins]

template = np.zeros(tracelength)
template[len(template)//2:len(template)//2 + 1000] = 1

fig, ax = plt.subplots(figsize=(12, 4))

ax.plot(np.arange(tracelength) / fs, template, color='k', linestyle='dashed', label="Square Pulse Template")
ax.set_ylabel("Amplitude [Arb.]")
ax.set_xlabel("Time [s]")
ax.legend(loc='upper right', edgecolor='k', framealpha=1)
ax.set_xlim(0, 30)
ax.tick_params(which='both', direction='in', right=True, top=True)
fig.tight_layout()

Now, let's create some white noise and add 10 pulses that are randomly placed throughout the trace.

In [ ]:
simulated_data = np.random.normal(scale=0.001, size=tracelength)

inds = np.random.choice(np.arange(tracelength) - tracelength//2, 10)
for val in inds:
    simulated_data += np.roll(template, val)

Plotting the subsequent simulated continuous data, we see that there are indeed about 10 pulses. Next, we'll convert this to a `splendaq` HDF5 file, which could then, e.g., be passed through the event building algorithms (see the event building tutorial).

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(np.arange(tracelength) / fs, simulated_data, color='k', label="Simulated Data")
ax.set_ylabel("Amplitude [Arb.]")
ax.set_xlabel("Time [s]")
ax.legend(loc='upper right', edgecolor='k', framealpha=1)
ax.set_ylim(-0.025, 1.15)
ax.set_xlim(0, 30)
ax.tick_params(which='both', direction='in', right=True, top=True)
fig.tight_layout()

Now, we will create the HDF5 file and supply all of the needed metadata for the file that is needed for eventbuilding.

Each metadata entry needed is detailed as:
 - **`data`**: the simulated data in shape (number of traces, number of channels, length of traces in bins)
 - **`channels`**: the names of each channel as a string
 - **`comment`**: some comment to pass to the data as a short description
 - **`fs`**: the digitization rate of the data in Hz
 - **`datashape`**: the shape of the data ndarray
 - **`eventindex`**: the index at which the event starts (this is continuous data, so it starts at zero)
 - **`eventnumber`**: the number of the event in increasing value (this is just zero, since there is just one event)
 - **`eventtime`**: the time of the beginning of the event in epoch time
 - **`seriesnumber`**: the seriesnumber of the event (format is YYYYMMDDhhmmss)
 - **`dumpnumber`**: if the data has been split into multiple files to reduce size, then this is which number of file
 - **`triggertime`**: the time of the trigger (since this is continuous, this is just the epoch time of the beginning of the trace)
 - **`triggertype`**: the type of trigger (0: randoms, 1: threshold trigger) continuous is treated as a long randomly triggered trace
 - **`parentseriesnumber`**: if the file was generated off of an existing file, the seriesnumber of that file. Set to `seriesnumber` if this is the original file
 - **`parenteventnumber`**: if the event was generated off of an existing event, the eventnumber of that event in the original file. Set to `eventnumber` if this is the original file.

We use the `splendaq.io.Writer` class to write the data, which has the single method `write_data`. Below, we show how to create this simulated continuous data and save it to a `splendaq` HDF5 file.

In [ ]:
start = datetime.now()

savename = "continuous_" + start.strftime("%Y%m%d_%H%M%S")
seriesnumber = int(start.strftime("%y%m%d%H%M%S"))
epochtime = start.timestamp()
comment = 'simulated_data, 10 pulses with amplitude 1'

data = simulated_data[None, None]
datashape = data.shape

FW = Writer(filename=f'./{savename}.h5')

FW.write_data(
    data=data,
    channels=[f'{ii}' for ii in range(datashape[1])],
    comment=comment,
    fs=fs,
    datashape=datashape,
    eventindex=[0] * datashape[0],
    eventnumber=np.arange(datashape[0]),
    eventtime=[epochtime] * datashape[0],
    seriesnumber=[seriesnumber] * datashape[0],
    dumpnumber=[1] * datashape[0],
    triggertime=[epochtime] * datashape[0],
    triggertype=[0] * datashape[0],
    parentseriesnumber=[seriesnumber] * datashape[0],
    parenteventnumber=np.arange(datashape[0]),
)

We have now written the file, let's read it with `splendaq.io.Reader` and plot the data to ensure it looks as expected.

In [ ]:
FR = sq.io.Reader(f'./{savename}.h5')

contdata, contmetadata = FR.get_data(include_metadata=True)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(np.arange(tracelength) / fs, contdata[0, 0], color='k', label="Simulated Data")
ax.set_ylabel("Amplitude [Arb.]")
ax.set_xlabel("Time [s]")
ax.legend(loc='upper right', edgecolor='k', framealpha=1)
ax.set_ylim(-0.025, 1.15)
ax.set_xlim(0, 30)
ax.tick_params(which='both', direction='in', right=True, top=True)
fig.tight_layout()

And it looks identical to what we thought we saved!